In [2]:
import random
import time
from collections import deque

# Define as dimensões do tabuleiro
N = 3

# Estrutura para armazenar um estado do puzzle
class EstadoPuzzle:
    def __init__(self, tabuleiro, x, y, profundidade, movimento=None):
        self.tabuleiro = tabuleiro
        self.x = x
        self.y = y
        self.profundidade = profundidade
        self.movimento = movimento  # Movimento que gerou esse estado

# Movimentos possíveis: Esquerda, Direita, Cima, Baixo
linha = [0, 0, -1, 1]
coluna = [-1, 1, 0, 0]
nomes_movimentos = ['Esquerda', 'Direita', 'Cima', 'Baixo']

# Função para verificar se um estado é o estado objetivo
def estado_objetivo(tabuleiro):
    objetivo = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    return tabuleiro == objetivo

# Função para verificar se um movimento é válido
def movimento_valido(x, y):
    return 0 <= x < N and 0 <= y < N

# Função para imprimir o tabuleiro
def imprimir_tabuleiro(tabuleiro):
    for linha in tabuleiro:
        print(' '.join(map(str, linha)))
    print("--------")

# Função para contar inversões no tabuleiro
def contar_inversoes(tabuleiro):
    lista_plana = [num for linha in tabuleiro for num in linha if num != 0]
    inversoes = sum(
        1 for i in range(len(lista_plana)) for j in range(i + 1, len(lista_plana))
        if lista_plana[i] > lista_plana[j]
    )
    return inversoes

# Função para verificar se o tabuleiro é solucionável
def e_solucionavel(tabuleiro):
    return contar_inversoes(tabuleiro) % 2 == 0 # Se o número de inversões for par, é soluci

def novo_tabuleiro_aleatorio():
    while True:
        numeros = list(range(N * N))
        random.shuffle(numeros)
        matriz = [numeros[i:i+N] for i in range(0, N * N, N)]
        
        if e_solucionavel(matriz):  # Verifica se o tabuleiro é solucionável
            return matriz

# Função para permitir que o usuário insira um tabuleiro inicial
def criar_tabuleiro_definido():
    print("Insira o tabuleiro linha por linha, inserindo os números separados por espaço.")
    print("Exemplo para a primeira linha: 1 2 3")
    tabuleiro = []
    for i in range(N):
        while True:
            try:
                linha = list(map(int, input(f"Inserir linha {i+1}: ").split()))
                if len(linha) != N:
                    raise ValueError
                tabuleiro.append(linha)
                break
            except ValueError:
                print("Entrada inválida. Por favor, insira 3 números separados por espaço.")
    return tabuleiro

# Função para salvar os resultados no arquivo
def salvar_resultados(movimentos, passos, tempo_decorrido, inicio):
    with open("solucaoBFS.txt", "w") as f:
        f.write(f"\nTotal de passos: {passos}\n")
        f.write(f"Tempo de processamento: {tempo_decorrido:.4f} segundos\n")
        f.write("Tabuleiro inicial:\n")
        
        for linha in inicio:
            f.write(" ".join(map(str, linha)) + "\n")  # Escreve o tabuleiro inicial corretamente
            
        f.write("--------\n")
        f.write("Sequência de movimentos:\n")
        
        for movimento, tabuleiro in movimentos:
            f.write(f"{movimento}:\n")
            for linha in tabuleiro:
                f.write(" ".join(map(str, linha)) + "\n")
            f.write("--------\n")

# Função BFS (busca em largura) para resolver o problema do puzzle 8
def resolver_puzzle_bfs(inicio, x, y):
    tempo_inicio = time.time()
    
    fila = deque()
    visitados = set()
    movimentos = []
    
    fila.append(EstadoPuzzle(inicio, x, y, 0))
    visitados.add(tuple(map(tuple, inicio)))

    while fila:
        atual = fila.popleft()

        if estado_objetivo(atual.tabuleiro):
            tempo_fim = time.time()
            tempo_decorrido = tempo_fim - tempo_inicio
            print(f'Estado objetivo alcançado na profundidade {atual.profundidade}')
            movimentos.append(('Objetivo Alcançado', atual.tabuleiro))
            salvar_resultados(movimentos, len(movimentos), tempo_decorrido, inicio)

            return

        for i in range(4):
            novo_x = atual.x + linha[i]
            novo_y = atual.y + coluna[i]

            if movimento_valido(novo_x, novo_y):
                novo_tabuleiro = [linha[:] for linha in atual.tabuleiro]
                novo_tabuleiro[atual.x][atual.y], novo_tabuleiro[novo_x][novo_y] = novo_tabuleiro[novo_x][novo_y], novo_tabuleiro[atual.x][atual.y]

                if tuple(map(tuple, novo_tabuleiro)) not in visitados:
                    visitados.add(tuple(map(tuple, novo_tabuleiro)))
                    movimento_nome = nomes_movimentos[i]
                    movimentos.append((movimento_nome, novo_tabuleiro))
                    fila.append(EstadoPuzzle(novo_tabuleiro, novo_x, novo_y, atual.profundidade + 1, movimento_nome))

    print('Nenhuma solução encontrada (BFS atingiu o limite de profundidade)')

# Código principal
if __name__ == '__main__':
    print("Escolha uma opção:")
    print("1. Tabuleiro aleatório solucionável")
    print("2. Definir um tabuleiro manualmente")
    
    escolha = input("Escolha uma opção:\n1 - Inserir tabuleiro manualmente\n2 - Gerar tabuleiro aleatório\nOpção: ")

    if escolha == '1':
        inicio = criar_tabuleiro_definido()
    elif escolha == '2':
        inicio = novo_tabuleiro_aleatorio()
    else:
        print("Opção inválida. Saindo...")
        exit()

    # Encontrar a posição do espaço vazio (0)
    x, y = next((i, j) for i in range(N) for j in range(N) if inicio[i][j] == 0)

    print('Estado Inicial:')
    imprimir_tabuleiro(inicio)

    resolver_puzzle_bfs(inicio, x, y)


Escolha uma opção:
1. Tabuleiro aleatório solucionável
2. Definir um tabuleiro manualmente


Insira o tabuleiro linha por linha, inserindo os números separados por espaço.
Exemplo para a primeira linha: 1 2 3
Estado Inicial:
5 3 1
7 6 4
2 8 0
--------
Estado objetivo alcançado na profundidade 24
